<a href="https://colab.research.google.com/github/Neptune-Trojans/LLMs/blob/main/LLM_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install datasets
%pip install transformers -U

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

In [ ]:
# Define the model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Step 1: Extend the position embeddings to 16,384
def expand_position_embeddings(model, new_max_length=16384):
    old_position_embeddings = model.model.config.max_position_embeddings
    old_max_length, embedding_dim = old_position_embeddings.weight.shape

    # Create new position embeddings with extended size
    new_position_embeddings = torch.nn.Embedding(new_max_length, embedding_dim)
    new_position_embeddings.weight.data[:old_max_length] = old_position_embeddings.weight.data
    new_position_embeddings.weight.data[old_max_length:] = old_position_embeddings.weight.data.mean(dim=0)

    # Replace the old position embeddings in the model
    model.transformer.wpe = new_position_embeddings
    model.config.n_positions = new_max_length
    model.config.n_ctx = new_max_length

In [ ]:
# Expand the position embeddings
expand_position_embeddings(model, new_max_length=200)

# Step 2: Adjust the tokenizer's model max length
tokenizer.model_max_length = 16384

# Step 3: Prepare the dataset
# Example: Load a dataset for fine-tuning (can be replaced with your specific dataset)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

In [ ]:
print(model.model.config.max_position_embeddings)